In [ ]:
!pip install tensorflow==2.10
!pip install mtcnn

In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import tensorflow as tf
print(tf.__version__)

2.10.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Define paths

In [ ]:
IMAGE_HOME = './drive/MyDrive/data'
LABEL_HOME = './drive/MyDrive/data/label'
DS_HOME = './drive/MyDrive/data/saved_data'

### Encode features
- gender: 0/1
- age: one-hot encoding
- age_new: as discussed in the report, turn it into 8 binary classifications of 'is age of this image older than {age_class}'. For example, people in (8, 12) are also older than (4, 6), (0, 2), so vector would be `[[1],[1],[1],[0],[0],[0],[0],[0]]`

In [ ]:
GENDER_DICT = {'m': 0, 'f': 1}
AGE_DICT = {
    '(0, 2)': [1,0,0,0,0,0,0,0],
    '(4, 6)': [0,1,0,0,0,0,0,0],
    '(8, 12)': [0,0,1,0,0,0,0,0],
    '(15, 20)': [0,0,0,1,0,0,0,0],
    '(25, 32)': [0,0,0,0,1,0,0,0],
    '(38, 43)': [0,0,0,0,0,1,0,0],
    '(48, 53)': [0,0,0,0,0,0,1,0],
    '(60, 100)': [0,0,0,0,0,0,0,1],
}
AGE_DICT_NEW = {
    '(0, 2)': [[1],[0],[0],[0],[0],[0],[0],[0]],
    '(4, 6)': [[1],[1],[0],[0],[0],[0],[0],[0]],
    '(8, 12)': [[1],[1],[1],[0],[0],[0],[0],[0]],
    '(15, 20)': [[1],[1],[1],[1],[0],[0],[0],[0]],
    '(25, 32)': [[1],[1],[1],[1],[1],[0],[0],[0]],
    '(38, 43)': [[1],[1],[1],[1],[1],[1],[0],[0]],
    '(48, 53)': [[1],[1],[1],[1],[1],[1],[1],[0]],
    '(60, 100)': [[1],[1],[1],[1],[1],[1],[1],[1]],
}

In [ ]:
def encode_label(gender, age):
  return GENDER_DICT.get(gender), AGE_DICT.get(age)

In [ ]:
def encode_label_new(gender, age):
  return GENDER_DICT.get(gender), AGE_DICT_NEW.get(age)

### Some age labels are wrong, correct them by classifying them to adjacent class

In [ ]:
def update_labels():
  for i in range(5):
    fname = 'fold_frontal_'+str(i)+'_data.txt'
    fpath = os.path.join(LABEL_HOME, fname)
    df = pd.read_csv(fpath, sep = '\t')
    df.loc[df['age'] == '(8, 23)', 'age'] = '(8, 12)'
    df.loc[df['age'] == '(38, 42)', 'age'] = '(38, 43)'
    df.loc[df['age'] == '(38, 48)', 'age'] = '(38, 43)'
    df.loc[df['age'] == '(27, 32)', 'age'] = '(25, 32)'
    df.loc[df['age'] == '32', 'age'] = '(25, 32)'
    df.loc[df['age'] == '46', 'age'] = '(48, 53)'
    df.loc[df['age'] == '42', 'age'] = '(38, 43)'
    df.loc[df['age'] == '34', 'age'] = '(25, 32)'
    df.loc[df['age'] == '29', 'age'] = '(25, 32)'
    df.loc[df['age'] == '2', 'age'] = '(0, 2)'
    df.loc[df['age'] == '56', 'age'] = '(48, 53)'
    df.loc[df['age'] == '57', 'age'] = '(60, 100)'
    df.loc[df['age'] == '23', 'age'] = '(25, 32)'
    df.loc[df['age'] == '36', 'age'] = '(38, 43)'
    df.loc[df['age'] == '45', 'age'] = '(38, 43)'
    df.loc[df['age'] == '13', 'age'] = '(8, 12)'
    df.loc[df['age'] == '22', 'age'] = '(15, 20)'
    df.loc[df['age'] == '58', 'age'] = '(60, 100)'
    df.loc[df['age'] == '55', 'age'] = '(60, 100)'
    df.loc[df['age'] == '35', 'age'] = '(38, 43)'
    df.loc[df['age'] == '3', 'age'] = '(0, 2)'
    new_fname = 'fold_frontal_'+str(i)+'_update.txt'
    new_fpath = os.path.join(LABEL_HOME, new_fname)
    df.to_csv(new_fpath, sep='\t', index=False)

### Process image:
- We use MTCNN to crop images to the person's face only
- Then resize it to required size ([224, 224] for ResNet34)
- Then normalize it by pixel/255

In [ ]:
def process_img(path, required_size=(224, 224)):
  pixels = plt.imread(path)
  detector = MTCNN()
  # detect face inside a box
  results = detector.detect_faces(pixels) 
  # if a face is detected
  if len(results) > 0:
    # get face bounding box
    x1, y1, width, height = results[0]['box'] 
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]

    image = Image.fromarray(face)
    image = image.resize(required_size) # resize to 224,224 because the model requires it to be that way 
    image.save(path[:-4]+'_cropped.jpg') # rename it to 'old_name_crop.jpg'
    face_array = np.asarray(image)
    return face_array/255 # normalize it
  else:
    return None

In [ ]:
# when create new dataset with new age encoding, directly used the cropped images hence only need to normalize it
def process_img_new(path, required_size=(224, 224)):
  return plt.imread(path)/255

### Create dataset for each fold

In [ ]:
def read_save():
  for i in range(1, 5):
    fold_X = []
    fold_gender = []
    fold_age = []

    # read each fold's images and labels information
    f_name = 'fold_frontal_' + str(i) + '_update.txt'
    f = os.path.join(LABEL_HOME, f_name)
    df = pd.read_csv(f, sep='\t')

    # for each row (image), process image, encode age and gender
    for index, row in df.iterrows():
      # find image
      image_name = 'landmark_aligned_face.' + str(row['face_id']) + '.' + row['original_image']
      image_path = os.path.join(IMAGE_HOME, row['user_id'], image_name)

      # process image
      image = process_img(image_path)

      # encode age and gender
      gender, age = encode_label(row['gender'], row['age'])

      # ignore this image since label might be wrong or no face detected
      if gender is None or age is None or image is None: 
        continue

      fold_X.append(image)
      fold_gender.append(gender)
      fold_age.append(age)
    
    # combine all within this fold and save as tf dataset
    fold_X = tf.convert_to_tensor(fold_X)
    fold_gender = tf.convert_to_tensor(fold_gender)
    fold_age = tf.convert_to_tensor(fold_age)
    path = os.path.join(DS_HOME, 'cv_fold'+str(i))
    ds = tf.data.Dataset.from_tensor_slices((fold_X, {'gender_output': fold_gender, 'age_output': fold_age}))
    tf.data.Dataset.save(ds, path)

In [ ]:
def read_save_new():
  for i in range(1,5):
    fold_X = []
    fold_gender = []
    fold_age = []

    # read each fold's images and labels information
    f_name = 'fold_frontal_' + str(i) + '_update.txt'
    f = os.path.join(LABEL_HOME, f_name)
    df = pd.read_csv(f, sep='\t')

    # for each row (image), process image, encode age and gender
    for index, row in df.iterrows():
      # find image
      image_name = 'landmark_aligned_face.' + str(row['face_id']) + '.' + row['original_image']
      image_path = os.path.join(IMAGE_HOME, row['user_id'], image_name[:-4]+'_cropped.jpg')

      # process image
      # some images don't have faces detected by MTCNN and hence don't have the cropped image. Add condition check to skip them
      if os.path.exists(image_path):
        image = process_img_new(image_path)
      else:
        image = None
        
      # encode age and gender
      gender, age = encode_label_new(row['gender'], row['age'])

      # ignore this image since label might be wrong or no face detected
      if gender is None or age is None or image is None:
        continue
        
      fold_X.append(image)
      fold_gender.append(gender)
      fold_age.append(age)
    
    # combine all within this fold and save as tf dataset
    fold_X = tf.convert_to_tensor(fold_X)
    fold_gender = tf.convert_to_tensor(fold_gender)
    fold_age = tf.convert_to_tensor(fold_age)
    path = os.path.join(DS_HOME, 'cv_fold'+str(i)+'_new')

    '''
    fold_age = [[[1],[0],[0],[0],[0],[0],[0],[0]], # age encoding for first example
                [[1],[1],[0],[0],[0],[0],[0],[0]], # age encoding for second example
                ...]
    hence, first column is label for 'age_group1'
    '''
    ds = tf.data.Dataset.from_tensor_slices((fold_X, {'gender_output': fold_gender, 
                                                      'age_group1': fold_age[:, 0],
                                                      'age_group2': fold_age[:, 1],
                                                      'age_group3': fold_age[:, 2],
                                                      'age_group4': fold_age[:, 3],
                                                      'age_group5': fold_age[:, 4],
                                                      'age_group6': fold_age[:, 5],
                                                      'age_group7': fold_age[:, 6],
                                                      'age_group8': fold_age[:, 7],}))
    tf.data.Dataset.save(ds, path)

In [ ]:
update_labels()

In [ ]:
read_save()

In [ ]:
read_save_new()